# ETL GCP Range of time 
Este script está enfocado en la carga de datos de todo un rango de tiempo (a diferencia del ETL GCP Daily  que solo sube los registros de un día en particular)

**PARAMTROS PARA LA CARGA DE DATOS** 

In [1]:
# 1) UNEs a consultar 
UNEs = ('D001', 'D002', 'D003', 'D004')

# 2) Credenciales SAP 
direccion_IP_SAP = "jdbc:sap://192.168.198.234:30241/" # Acá contiene  más elemento, no solo el IP del host, acá un desloce de los elementos: 

# - jdbc: Indica que se está utilizando Java Database Connectivity (JDBC) para la conexión.
# - sap: Especifica el tipo de base de datos al que te estás conectando, en este caso, SAP HANA.
# - 192.168.198.234: Es la dirección IP del host donde se encuentra la base de datos SAP HANA.
# - 30241: Es el puerto a través del cual se establece la conexión a la base de datos.

usuario_SAP  = "QLIKSAP"
password_SAP = "20S@pqlik"
driver_SAP = "com.sap.db.jdbc.Driver"


# 3) Credenciales SQL Sever (Eroute) 
# Configuración para la conexión a SQL Server
direccion_IP_SQL_SERVER = "jdbc:sqlserver://192.168.0.192;encrypt=false;databaseName=ASR;" # Muy importatante, vease como tengo el parametro encrypt=false, sin esto, no me dejaba conectarme a la BD 

usuario_SQL_Server = "consulta" 
password_SQL_Server  = "Consult@"
driver_SQL_Server = "com.microsoft.sqlserver.jdbc.SQLServerDriver" 


# 4) Parametros de GCP. 
# La exportación de los datos se hará a GCP  a 2 distintas repositorios de datos los cuales son Google Cloud Storage y BigQuery, en esta sección vamos a idnetificar los paámetros que se requiere para la correcta exportación de datos 
bucket = "gs://sap_eroute"
proyecto = "ciencia-de-datos-398421" # Este es el proyecto global de GCP
conjunto_datos_BQ = "SAP_Eroute"  # Esto es el conjunto de datos (equivalente a mi base de datos) en Bigquery. 
modo_escritura  = "WRITE_TRUNCATE"   # WRITE_APPEND va a agregar datos a los ya existentes, WRITE_TRUNCATE reemplaza los datos existentes por los nuevos en BQ 

**IMPORTACION DE LIBRERIAS**

In [2]:
import datetime
import pandas as pd
from google.cloud import bigquery # Interacción con bigqueyr 
from google.oauth2 import service_account # Autentificación en GCP 
import os 
from pyspark.sql.functions import col, to_date

from pyspark.pandas.config import set_option

C:\Users\OMBARRAZA\miniconda3\envs\Spark\Lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


 **SELECCION DE FECHAS DE INICIO Y FIN DE LA CARGA**
 Este script está pensado para ser un script que obtenga los registros solo de un día (el día de ayer) 

In [3]:
# 1) Obtengo la fecha actual 
fecha_actual  = datetime.date.today()

# 2) La paso a un objeto de pandas date time 
fecha_actual = pd.to_datetime(arg=fecha_actual) # Dado que el objeto de entrada es ya un objeto date time, no es neceario que le  indique el formato 
print("Fecha actual:", fecha_actual)

# 3) Obtengo la fecha del día de ayer 
dia = pd.to_timedelta(arg=1, unit="d") 
fecha_ayer = fecha_actual - dia
print("Fecha ayer:", fecha_ayer)

Fecha actual: 2024-03-06 00:00:00
Fecha ayer: 2024-03-05 00:00:00


In [4]:
# Todo los almacenams en tipo string 

current_year=  str(fecha_actual.year)
current_month =  str(fecha_actual.month)
current_day = str(fecha_actual.day)

# Hacemos una pequeña adaptación de los datos para le fornmato de la columna de DIA CLAVE dada de alta como STRING en SAP
if len(current_month) < 2 :
    current_month = '0' + current_month

yesterday_year =   str(fecha_ayer.year)
yesterday_month = str(fecha_ayer.month) 
yesterday_day =  str(fecha_ayer.day)

# Hacemos una pequeña adaptación de los datos para le fornmato de la columna de DIA CLAVE dada de alta como STRING en SAP  
if len(yesterday_month) < 2 :
    yesterday_month = '0' + yesterday_month

fecha_ayer = yesterday_day + "/" + yesterday_month + "/" + yesterday_year 

print("Año hoy:", current_year)
print("Mes hoy:", current_month)
print("Dia hoy:", current_day)

print("Año ayer:", yesterday_year)
print("Mes ayer:", yesterday_month)
print("Dia ayer:", yesterday_day)

Año hoy: 2024
Mes hoy: 03
Dia hoy: 6
Año ayer: 2024
Mes ayer: 03
Dia ayer: 5


In [5]:
# Primer formato de fecha de ayer 
yesterday_date = yesterday_year + "-" + yesterday_month + "-" + yesterday_day      
# Segundo formato de  fecha de ayer
yesterday_date_second_format  = yesterday_year + yesterday_month + yesterday_day # Esto es año, mes y dia 

print("primer formato:", yesterday_date)
print("Segundo formato:", yesterday_date_second_format)

primer formato: 2024-03-5
Segundo formato: 2024035


In [6]:
fecha_inicio = "2021-08-20" # Te puedes remontar hasta la fecha: Año, mes y día (original 2021-08-20)

## 1) Autentificación en GCP 
La autentificación servira para poder hacer las tranferencias de datos desde hacía Cloud Storage y de Cloud Storage hacia Bigquery 

In [7]:
# Dirección de trabajo 
direccion = os.getcwd()

# Autentificación con las credenciales, nota: Estas credenciales ya contienen los permisos para interactual en gran medida con Google Cloud Storage y Bigquery 
bqcreds = service_account.Credentials.from_service_account_file(filename= direccion + "\\Credenciales\\Auth_Storage_BigQuery.json",  scopes = ['https://www.googleapis.com/auth/cloud-platform'])

# Obtencion del cliente de biquery que permitira interactual con bigquery 
client = bigquery.Client(credentials=bqcreds, project=bqcreds.project_id) # El cliente de bigquery es necesario para poder escribr en las tablas de BQ
client

## 2) Inicio de sesión en Apache Spark
El inicio de sesión de mi aplicación de Apache Spark es muy relevant, aqui le indicaremos los siguinetes aspectos de la sesión: 

- Proporcionar los conectores JDBC: Tenemos que descargar los controladores necesarioas para hacer las conexiones y brindale la dirección de dichas credenciales  
- Definición del cluster manager 
- RAM para cada executor (Exeutor memory):


**PENDIENET** profundizar más en todos los aspectos que puedo configurar y mencionar la configuración por degault que se tiene, ahorita simplemente le aumentare la memoria disponible que puede consumir 

 

In [8]:
import findspark # Se importa antes de cualquier inicialización  o impotación de spark 
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName(name="ETL GCP Range Time") \
    .config("spark.executor.memory", "12g") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .config("spark.hadoop.mapreduce.fileoutputcommitter.marksuccessfuljobs", "false") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", r"C:\Users\OMBARRAZA\Documents\Sello Rojo\Apache Spark ETL\Credenciales\Auth_Storage_BigQuery.json") \
    .config("spark.jars", r"C:\Users\OMBARRAZA\Documents\Sello Rojo\Apache Spark ETL\drivers\sap\ngdbc-2.18.11.jar," +
            r"C:\Users\OMBARRAZA\Documents\Sello Rojo\Apache Spark ETL\drivers\sqljdbc_12.2\enu\mssql-jdbc-12.2.0.jre8.jar," +
            r"C:\Users\OMBARRAZA\miniconda3\envs\Apache_Spark\Lib\site-packages\pyspark\jars\gcs-connector-hadoop3-latest.jar") \
    .getOrCreate()

spark

In [9]:
configuracion = spark.sparkContext.getConf()

print("Memoria del ejecutor:", configuracion.get("spark.executor.memory"))

Memoria del ejecutor: 12g


## 3) Conexión a las bases de datos 

### 3.1) Autentificación a SAP HANA 

Usaremos los parametros previamente establecidos 

In [10]:
# 1) Le damos la dirección IP donde se encuentra el host de SAP Hana 
database_url_sap = direccion_IP_SAP

# 2) Le damos la autentificación 
database_properties_sap = {
    "user": usuario_SAP,
    "password": password_SAP,
    "driver": driver_SAP
}

### 3.2) Autentificación SQL Server 

In [11]:
# Configuración para la conexión a SQL Server
database_url_sql_server = direccion_IP_SQL_SERVER

database_properties_sql_server = { 
          "user": usuario_SQL_Server, 
          "password": password_SQL_Server, 
          "driver": driver_SQL_Server
}

## 4) Descarga e inspección de datos 
La descarga de datos se hace para ciertos aspetros de la operación de interés. Los más relevantes son: 
 
1) **Agenda Actual:** Los datos vinculados a la agenda son muy relevantes, estos nos permite observar cual es la agenda actual de cada vendedor y compararlo con las visitas reales que ha tenido. Con estos 2 conjuntos de datos podemos observar cual es el grado de cumplimiento de la agenda de cada vendedor para cada día  y a nivel semanal
2)  **Agenda Historica:**

PONER QUE DATOS VAMOS A DESCARGAR!! 

### 4.1) Agenda actual (clientes activos), desde SAP  

**Estos son los clientes activos actuales**  

La agenda actual la sacaremos de **SAP Hana**, para construir la agenda actual de las rutas tenemos que consultar las siguientes tablas: 

- **KNA1:** Contiene información acerca de los de clientes registrados en la empresa, asi como su estado (activo o inactivo) 
- **KNVV:** Contiene los datos de las rutas (que rutas atienden a que cliente)  y a que UNEs pertenece 


Dado que tenemos estas 2 tablas y requereimos información de ambas, vamos a hacer una unión de tablas

**COLUMNAS DE KNA1:**
- **KUNNR:** ID único de cada cliente, le dimos el alias de **ClienteClave**
- **NAME1**: Nomnbre registrado del cliente, le dimos el alias de **Nombre**
- **ORT02:** Es la localidad donde está registrado el cliente, le dimos el alias de **Localidad**
- **ORT01:** Es la colonia del cliente, se le dio el alias de **Colonia** 
- **PSTLZ:** Es el codigo postal del domicilio registrado del cliente, se le dio el alias de **CodigoPostal**
- **REGIO**: Es el estado de la republica, se le dio el alias de **Estado**
- **STRAS:** Es la calle y nbúmero del cliente, se le dio el alias de **Calle**
- **BAHNE:** Es la hora de apertura del cliente, se le dio el alias de **HoraApertura**
- **BAHNS** Es la hora de cierre del cliente, se le dio el alias de **HoraCierre**
- **LZONE:** Es la UNE a la que pertenece el cliente, le dimos el alías de **UNE**
- **LOEVM:** Es el estatus del cliente, le dimos el alias de **STATUS**  
- **KATR1:** Corresponde al día de visita lunes, se le dio el alias de **Lunes**
- **KATR2:** Corresponde al día de visita martes, se le dio el alias de **Martes**
- **KATR3:** Corresponde al día de visita miercoles, se le dio el alias de **Miercoles**
- **KATR4:** Corresponde al día de visita jueves, se le dio el alias de **Jueves**
- **KATR5:** Corresponde al día de visita viernes, se le dio el alias de **Viernes**
- **KATR6:** Corresponde al día de visita sábado, se le dio el alias de **Sabado**
- **KATR7:** Corresponde al día de visita domingo, se le dio el alias de **Domingo**



In [18]:
# Declaración del query  
query_clientes = f"""
SELECT 
KUNNR as ClienteClave, NAME1 as Nombre, ORT02 as Localidad,  ORT01 as Colonia, PSTLZ as CodigoPostal, 
REGIO as Estado, STRAS as Calle,  BAHNE as HoraApertura, BAHNS as HoraCierre, LZONE as UNE, LOEVM as STATUS, 
KATR1 as Lunes, KATR2 as Martes, KATR3 as Miercoles, KATR4 as Jueves, KATR5 as Viernes,  KATR6 as Sabado, 
KATR7 as Domingo
FROM SAPABAP1.KNA1
WHERE LZONE  IN {UNEs}
AND LOEVM = ''
AND  KATR1  != ''
AND  KATR2 != ''
AND  KATR3 != ''
AND  KATR4 != ''
AND  KATR5 != ''
AND  KATR6 != ''
"""

# IMPORTANTE: Tengo que darle el query en una cadena dentro de paranetesis 
query_clientes =  f"({query_clientes}) as query_alias"

**NOTA IMPORTANTE: SELECCION DE CLIENTES ACTIVOS** 
Los clientes activos o inactivos se pueden diferenciar con la columna "LOEVM" la cual tiene el nombre comun de "Petición de borrado", esta columna si está marcada con una X significa que ese cliente esta inactivo. Por lo tanto nos quedaremos solo con los clientes con esa columna vacia.

Adicionalmente me quedo solo con los clientes que tienen una frecuencia de visita, la frecuencina de visita es un valor binario que está en las columnas KATR1, KATR2, KATR3 etc. Son los días de la semana y se pone un 1 cuando se visita ese día de la semana 

In [19]:
%%time
# 1) Leer datos en un DataFrame
agenda_actual_clientes_activos = spark.read.jdbc(url=database_url_sap, table=query_clientes, properties=database_properties_sap)

#print("Tipo de objeto:", type(agenda_actual_clientes_activos))
# Notese que cuando lees un archivo por default se guarda en un spark data frame, yo lo convertire a un spark pandas DF 

# Lo paso a un Spark Pandas DF 
agenda_actual_clientes_activos = agenda_actual_clientes_activos.to_pandas_on_spark()
#print("Tipo de objeto:", type(agenda_actual_clientes_activos)) # Vease que ahora cambio del tipo de objeto

# FALTA ALGO, quiero integrar el TIPO de clientes !!!, agregar eso 

CPU times: total: 0 ns
Wall time: 659 ms


C:\Users\OMBARRAZA\miniconda3\envs\Spark\Lib\site-packages\pyspark\sql\dataframe.py:5723: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(


**GENERACION DE VISITAS SEMANALES POR CLIENTE**

In [20]:
# 1) Cambiamos el tipo de variables 
agenda_actual_clientes_activos["LUNES"] = agenda_actual_clientes_activos["LUNES"].astype(int)
agenda_actual_clientes_activos["MARTES"] = agenda_actual_clientes_activos["MARTES"].astype(int)
agenda_actual_clientes_activos["MIERCOLES"] = agenda_actual_clientes_activos["MIERCOLES"].astype(int)
agenda_actual_clientes_activos["JUEVES"] = agenda_actual_clientes_activos["JUEVES"].astype(int)
agenda_actual_clientes_activos["VIERNES"] = agenda_actual_clientes_activos["VIERNES"].astype(int)
agenda_actual_clientes_activos["SABADO"] = agenda_actual_clientes_activos["SABADO"].astype(int)
agenda_actual_clientes_activos["DOMINGO"] = agenda_actual_clientes_activos["DOMINGO"].astype(int)

In [21]:
# 1) Ejecución de la suma 
agenda_actual_clientes_activos['Frecuencia_Semanal'] = agenda_actual_clientes_activos["LUNES"] + agenda_actual_clientes_activos["MARTES"] +  agenda_actual_clientes_activos["MIERCOLES"] + agenda_actual_clientes_activos["JUEVES"] + agenda_actual_clientes_activos["VIERNES"] + agenda_actual_clientes_activos["SABADO"]    

In [22]:
# Filtro de clientes activos realmente con visita programaeda  
agenda_actual_clientes_activos = agenda_actual_clientes_activos.query("Frecuencia_Semanal  > 0 ") # Nos quedamos con los clientes que tienen mayor a 0  visitas por semana 

agenda_actual_clientes_activos.reset_index(drop=True, inplace=True)

#### 4.1.1) Extracción de la tabla clientes - rutas
**EXTRACCION DE LA TABLA CLIENTES - RUTAS**

NOTAS:
- **KUNNR:** Es el ID del cliente 
- **VWERK:** Es la UNE al que pertenece 
- **VKGRP:** Es el Ruta

In [17]:
%%time 
# 1) Aislo la columna  de interés
ID_clientes_activos = agenda_actual_clientes_activos[["CLIENTECLAVE"]]

# 2) Valors unicos 
ID_clientes_activos = ID_clientes_activos["CLIENTECLAVE"].unique() # Esto me generará una spark pandas Serie

# 3) Lo paso el spark serie  a un numpy array 
ID_clientes_activos = ID_clientes_activos.to_numpy()

# 4) Lo paso a una tupla 
ID_clientes_activos = tuple (ID_clientes_activos)
ID_clientes_activos

C:\Users\OMBARRAZA\miniconda3\envs\Spark\Lib\site-packages\pyspark\pandas\utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


CPU times: total: 46.9 ms
Wall time: 1.58 s


('0458450000',
 '0512880000',
 '0637190000',
 '1840970001',
 '0332510000',
 '0491630000',
 '0635690000',
 '0494550000',
 '0444810000',
 '0494600000',
 '0324870000',
 '1924460001',
 '0694440001',
 '0355360001',
 '0513730000',
 '1818470001',
 '1454860001',
 '1697370001',
 '1907610001',
 '0340660001',
 '1362440001',
 '1869200001',
 '1850570001',
 '0330660000',
 '1874750001',
 '1478950001',
 'E3328',
 'E15510',
 'E13357',
 'E13188',
 'E12627',
 'E14929',
 'E11278',
 'E8182',
 '0442930000',
 '0447190000',
 '0435750000',
 '0466610000',
 '0610000000',
 '1909070001',
 '0352800000',
 '1478830001',
 '1478250001',
 '0454350000',
 '1908740001',
 '1596230001',
 '1481130001',
 '0351290000',
 '0624990000',
 '0516830000',
 '0425560000',
 '0525410000',
 '0500970000',
 '0464710000',
 '0464830000',
 '0321770000',
 '0310110000',
 '0466510000',
 '1179490000',
 '0659890000',
 '0362200000',
 '0358810000',
 '0356780000',
 '0479970000',
 '0506340000',
 '0504060000',
 '0466180000',
 '0420420000',
 '0401160000',

In [18]:
query_rutas_cliente = f"""
SELECT 
    KNVV.KUNNR as ClienteClave, 
    KNVV.VKGRP as RUTA, 
    KNVV.ERDAT as FECHA
FROM 
    SAPABAP1.KNVV 
INNER JOIN (
    SELECT 
        KUNNR, 
        MAX(TO_DATE(ERDAT, 'YYYYMMDD')) AS MaxDate
    FROM 
        SAPABAP1.KNVV
    GROUP BY 
        KUNNR
) AS LatestRecords
ON 
    KNVV.KUNNR = LatestRecords.KUNNR AND TO_DATE(KNVV.ERDAT, 'YYYYMMDD') = LatestRecords.MaxDate
WHERE  KNVV.SPART = 10  AND  KNVV.KUNNR IN {ID_clientes_activos}
ORDER BY 
    TO_DATE(KNVV.ERDAT, 'YYYYMMDD') DESC
"""

# IMPORTANTE: Tengo que darle el query en una cadena dentro de paranetesis 
query_rutas_cliente =  f"({query_rutas_cliente}) as query_alias"

In [19]:
# 1) Leer datos en un DataFrame
rutas_cliente = spark.read.jdbc(url=database_url_sap, table=query_rutas_cliente, properties=database_properties_sap)

# 2) Lo pasamos a un spark pandas DF 
rutas_cliente = rutas_cliente.to_pandas_on_spark()

C:\Users\OMBARRAZA\miniconda3\envs\Spark\Lib\site-packages\pyspark\sql\dataframe.py:5723: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(


**VINCULACION DE RUTAS A CLIENTES ACTUALES ACTIVOS**

In [20]:
agenda_actual_clientes_activos = agenda_actual_clientes_activos.merge(right=rutas_cliente, on="CLIENTECLAVE", how="left")

### 4.2) Agenda historica, desde Eroute   (SQL Server)

**SOLO AYER:** se almacenara solo la agenda del dia de ayer  
- La agenda historica la sacaremos de Eroute,  ..... EXPLICAR QUE ES, es la agenda histprica de cada cliente o de cada ruia o de ambas?
- **FALTA PONER EL SIGNIFICATO DE TODAS LAS COLUMNAS** 

In [24]:
%%time 

# 1) Alta del query, esto viene de SQL Server!!  
query_agenda_historica = f"""
select DiaClave, FrecuenciaClave, RUTClave, ClienteClave, Orden, ClaveCEDI 
from AgendaVendedor 
where ClaveCEDI in {UNEs} 
and cast(DiaClave as date) between '{fecha_inicio}' and '{yesterday_date}'
"""
# El formato de fechas es año, mes y día 

# 2) Modificación del query 
# IMPORTANTE: este es el formato para los queries en spark  
query_agenda_historica =  f"({query_agenda_historica}) as query_alias"

# 3) Lectura de datos a un Spark DF 
agenda_historica = spark.read.jdbc(url=database_url_sql_server, table=query_agenda_historica, properties=database_properties_sql_server)

CPU times: total: 0 ns
Wall time: 176 ms


**AGREGAR SEMANA , AÑO Y MES**

In [25]:
agenda_historica = agenda_historica.withColumn("DiaClave", to_date(agenda_historica.DiaClave, "dd/MM/yyyy"))
agenda_historica = agenda_historica.to_pandas_on_spark()

C:\Users\OMBARRAZA\miniconda3\envs\Spark\Lib\site-packages\pyspark\sql\dataframe.py:5723: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(


In [26]:
agenda_historica["Month"] = agenda_historica["DiaClave"].dt.month
agenda_historica["Year"] = agenda_historica["DiaClave"].dt.year
agenda_historica["Week"] = agenda_historica["DiaClave"].dt.week
agenda_historica["Day"] = agenda_historica["DiaClave"].dt.day


C:\Users\OMBARRAZA\miniconda3\envs\Spark\Lib\site-packages\pyspark\pandas\datetimes.py:128: FutureWarning: weekofyear and week have been deprecated.
  warnings.warn(


### 4.3) Visitas desde Eroute
**SOLO DE AYER:** Se capturas las visitas solo de ayer 

**EXPLICACION DE LAS COLUMNAS**:
- **VisitaClave:** ID Único de la visita 
- **DiaClave:** Fecha de la visita 
- **ClienteClave:** ID  único del cliente
- **RUTClave:** ID único de la ruta que realizo esa visita 
- **Numero:** Es la secuencia en la que se visito ese día 
- **FechaHoraInicial:** Fecha y hora de llegada al cliente
- **FechaHoraFinal:** Fecha y hora de salida de con el cliente 

In [24]:
from pyspark.sql.functions import date_format

# 1) Query de las visitas 
query_visitas = f"""
select Visita.VisitaClave, Visita.DiaClave, Visita.ClienteClave, Visita.RUTClave, Ruta.AlmacenID, Visita.Numero,
       Visita.FechaHoraInicial, Visita.FechaHoraFinal from Visita
inner join Ruta on Visita.RUTClave = Ruta.RUTClave
where cast(DiaClave as date )  between '{fecha_inicio}' and '{yesterday_date}'
"""
## El formato es dia/mes/año

# 2) Tranformación del formato 
query_visitas =  f"({query_visitas}) as query_alias"

# 3) Leer datos en un Spark DataFrame
visitas = spark.read.jdbc(url=database_url_sql_server, table=query_visitas, properties=database_properties_sql_server)

**AGREGAMOS SEMANA, MES y AÑO**

In [25]:
# 1) Lo pasamos  a un formato de fecha 
visitas = visitas.withColumn("DiaClave", to_date(visitas.DiaClave, "dd/MM/yyyy"))

# 2) Lo pasamos a un spark pandas DF 
visitas = visitas.to_pandas_on_spark()

# 3) Generamos el mes, año y semana apartir de la fecha con la funcion dt.. 
visitas["Month"] = visitas["DiaClave"].dt.month
visitas["Year"] = visitas["DiaClave"].dt.year
visitas["Week"] = visitas["DiaClave"].dt.week
visitas["Day"] = visitas["DiaClave"].dt.day

C:\Users\OMBARRAZA\miniconda3\envs\Spark\Lib\site-packages\pyspark\sql\dataframe.py:5723: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(
C:\Users\OMBARRAZA\miniconda3\envs\Spark\Lib\site-packages\pyspark\pandas\datetimes.py:128: FutureWarning: weekofyear and week have been deprecated.
  warnings.warn(


In [26]:
# 4) Tranformamos a un spark pandas DF 
## NOTA: Aparentemente no puedo pasar el spark DF a un spark pandas DF si hay una columna tipo datetime, tengo que pasarla a tipo string , acá una referencia: 
# https://stackoverflow.com/questions/76072664/convert-pyspark-dataframe-to-pandas-dataframe-fails-on-timestamp-column

#visitas = visitas.withColumn("FechaHoraInicial", date_format("FechaHoraInicial", "yyyy-MM-dd HH:mm:ss"))
#visitas= visitas.withColumn("FechaHoraFinal", date_format("FechaHoraFinal", "yyyy-MM-dd HH:mm:ss"))


# Ahoira si lo puedo pasar a un spark pandas DF 
#visitas = visitas.to_pandas_on_spark()

### 4.4) Ventas, cambios físicos y pedidos 

Toda esta información esta en las mismas 2 tablas, es una tabla cabecera (registros generales) y una tabla detalle con los registros a detalle. Estas 2 tablas son:  ZTSDCAB y ZTSDDET.

La clasificación del tipo de registros, es decir si se trato de una venta, una cambio físico, una carga real o un pedido se puede clasificar usando la columna **DOC_TYPE**. El tipo de registros que hay en esta columna son: 

- **ZPSR**: Registros de ventas
- **ZPSO**: Registros de paseo 
- **ZTO**: Cambios fisicos
- **ZPAS** Pedido planta 
- **ZPVE**: Pedido vendedor 



*EXPLICACION DE CLUMNAS DE LA TABLA ZTSDCAB:* 
- **PURCH_NO_C:** Es el ID único de la opreción (cualquier tipo de operación, venta, cambio etc.)  
- **DOC_TYPE:** Establece el tipo de operación 
- **SALES_OFF:** ID de la UNE 
- **SALES_GRP** ID de la ruta que ejecuto esa operación 
- **SALES_OFF:** Es la UNE
- **SALES_GRP:** Es la ruta 
- **PURCH_DATE:** Es la fecha en la que se hizo la operación
- **DIA_CALVE:** Es la fecha en la que se hizo la operación
- **FORMA_PAGO**: Categoria de forma de pago, se dejo con el mismo nombre 
- **TOTAL**: Total monetario

**GENERACION DE FECHAS CONCRETAS**
dado que la base de datos están mal declaradas las columnas como la de fecha y existen multiples registros que no contienen fechas validas  decidí hacer un filtro utilizando  el tiempo de interés (el rango de fechas) como una tuple en lugar de usar un between en el query. 

In [27]:
from datetime import datetime, timedelta

# Fechas de inicio y fin
fecha_inicio = datetime.strptime("7/7/2021", "%d/%m/%Y") # esto es dia, mes y año 
fecha_fin = datetime.strptime("5/3/2024", "%d/%m/%Y")

# Generar todas las fechas entre las dos fechas
delta = fecha_fin - fecha_inicio

fechas = [(fecha_inicio + timedelta(days=i)).strftime("%d/%m/%Y") for i in range(delta.days + 1)]

# Convertir la lista en tuple
fechas_tuple = tuple(fechas)

fechas_tuple

('07/07/2021',
 '08/07/2021',
 '09/07/2021',
 '10/07/2021',
 '11/07/2021',
 '12/07/2021',
 '13/07/2021',
 '14/07/2021',
 '15/07/2021',
 '16/07/2021',
 '17/07/2021',
 '18/07/2021',
 '19/07/2021',
 '20/07/2021',
 '21/07/2021',
 '22/07/2021',
 '23/07/2021',
 '24/07/2021',
 '25/07/2021',
 '26/07/2021',
 '27/07/2021',
 '28/07/2021',
 '29/07/2021',
 '30/07/2021',
 '31/07/2021',
 '01/08/2021',
 '02/08/2021',
 '03/08/2021',
 '04/08/2021',
 '05/08/2021',
 '06/08/2021',
 '07/08/2021',
 '08/08/2021',
 '09/08/2021',
 '10/08/2021',
 '11/08/2021',
 '12/08/2021',
 '13/08/2021',
 '14/08/2021',
 '15/08/2021',
 '16/08/2021',
 '17/08/2021',
 '18/08/2021',
 '19/08/2021',
 '20/08/2021',
 '21/08/2021',
 '22/08/2021',
 '23/08/2021',
 '24/08/2021',
 '25/08/2021',
 '26/08/2021',
 '27/08/2021',
 '28/08/2021',
 '29/08/2021',
 '30/08/2021',
 '31/08/2021',
 '01/09/2021',
 '02/09/2021',
 '03/09/2021',
 '04/09/2021',
 '05/09/2021',
 '06/09/2021',
 '07/09/2021',
 '08/09/2021',
 '09/09/2021',
 '10/09/2021',
 '11/09/20

#### 4.4.1) Tabla de ventas, cambios y pedidos 

In [28]:
query_general_ventas_pedidos_y_cambios = f"""
select ZTSDCAB.PURCH_NO_C, ZTSDCAB.DOC_TYPE, ZTSDCAB.SALES_OFF as UNE, ZTSDCAB.SALES_GRP as RUTA, 
ZTSDCAB.PURCH_DATE, ZTSDCAB.DIA_CLAVE, ZTSDCAB.FORMA_PAGO, 
ZTSDDET.MATERIAL, ZTSDDET.TARGET_QTY, ZTSDDET.COND_VALUE as TOTAL_DETALLE  
from SAPABAP1.ZTSDCAB  
inner join SAPABAP1.ZTSDDET ON  SAPABAP1.ZTSDCAB.PURCH_NO_C =   SAPABAP1.ZTSDDET.PURCH_NO_C
WHERE ZTSDCAB.DOC_TYPE IN ('ZPSR', 'ZTO', 'ZPAS', 'ZPVE')
AND ZTSDCAB.DIA_CLAVE IN {fechas_tuple} 
AND  ZTSDCAB.SALES_OFF IN {UNEs}
limit 100000
"""

In [29]:
# 2) Tranformamos formato 
query_general_ventas_pedidos_y_cambios = f"({query_general_ventas_pedidos_y_cambios}) as query_alias"

# 3) Leer datos en un DataFrame
ventas_pedidos_y_cambios = spark.read.jdbc(url=database_url_sap, table= query_general_ventas_pedidos_y_cambios, properties=database_properties_sap)

**AGREGAMOS SEMANA, MES y AÑO**

In [30]:
# 1) Lo pasamos  a un formato de fecha 
ventas_pedidos_y_cambios = ventas_pedidos_y_cambios.withColumn("DIA_CLAVE", to_date(ventas_pedidos_y_cambios.DIA_CLAVE, "dd/MM/yyyy"))

# 2) Lo pasamos a un spark pandas DF 
ventas_pedidos_y_cambios = ventas_pedidos_y_cambios.to_pandas_on_spark()

# 3) Generamos el mes, año y semana apartir de la fecha con la funcion dt.. 
ventas_pedidos_y_cambios["Month"] = ventas_pedidos_y_cambios["DIA_CLAVE"].dt.month
ventas_pedidos_y_cambios["Year"] = ventas_pedidos_y_cambios["DIA_CLAVE"].dt.year
ventas_pedidos_y_cambios["Week"] = ventas_pedidos_y_cambios["DIA_CLAVE"].dt.week
ventas_pedidos_y_cambios["Day"] = ventas_pedidos_y_cambios["DIA_CLAVE"].dt.day

**AGREGAR COLUMNAS MAS LEGIBLES**

In [31]:
equivalencia_tipo_movimiento = {"ZPSR": "Ventas", 
                "ZTO": "Cambios_Fisicos", 
                "ZPAS": "Pedidos_Planta", 
                "ZPVE": "Pedido_Vendedor"}

equivalencia_metodo_pago ={
    "3": "Transferencia", 
    "1": "Efectivo", 
    "2": "Cheque",
    "A": "Parcialidades",
    "": "Sin información"
}

In [32]:
ventas_pedidos_y_cambios["TIPO_OPERACION"] = ventas_pedidos_y_cambios["DOC_TYPE"].map(arg = equivalencia_tipo_movimiento)
ventas_pedidos_y_cambios["METODO_PAGO"] = ventas_pedidos_y_cambios["FORMA_PAGO"].map(arg = equivalencia_metodo_pago)

### 4.5) Venta con prefactura 

In [ ]:
nota= "falta que agregue las ventas que no salen en Eroute, que son las ventas por medio de prefactura a cadenas grandes"

### 4.5) Carga real de productos 
Carga se refiere a la carga real efecutada en las unidaded (rutas). Es decir los productos que se terminaron subiendo a las unidades. Para esto tenemos que hacer las consultas a las siguientes 2 tablas:
  
- ZINVWMCAB y  ZINVWMDET   


_______

**GLOSARIO:** 

- **PURCH_NO_C:** Es el ID del movimiento, no se le puso ningún alias
-  **DOC_TYPE:** Tipo de documento, no se le dio alias 
- **SALES_OFF:** Centro de distribución se le dió el alias de **UNE** 
- **SALES_GRP:** Es la ruta, se le dio el alias de **RUTA**
- **PURCH_DATE** Fecha del movimiento, no se le dio un alias 
- **DIA_CLAVE**: Fecha del movimiento 

**NOTA MUY IMPORTANTE:** 
- Las fechas vienen en la fecha de CARGA, recordemos que la carga se hace un día de trabajo antes
- En la columna de DOCTYPE estas son los significados de cada catregoria: 
CA = Cargas 
PF = Prefacturas 
RE = Recargas    




In [33]:
# Fechas de inicio y fin
fecha_inicio = datetime.strptime("2021/7/15", "%Y/%m/%d") # essto es año, mes y dia 
fecha_fin = datetime.strptime("2024/03/5", "%Y/%m/%d")

# Generar todas las fechas entre las dos fechas
delta = fecha_fin - fecha_inicio

fechas = [(fecha_inicio + timedelta(days=i)).strftime("%Y%m%d") for i in range(delta.days + 1)]

# Convertir la lista en tuple
fechas_tuple = tuple(fechas)
fechas_tuple

('20210715',
 '20210716',
 '20210717',
 '20210718',
 '20210719',
 '20210720',
 '20210721',
 '20210722',
 '20210723',
 '20210724',
 '20210725',
 '20210726',
 '20210727',
 '20210728',
 '20210729',
 '20210730',
 '20210731',
 '20210801',
 '20210802',
 '20210803',
 '20210804',
 '20210805',
 '20210806',
 '20210807',
 '20210808',
 '20210809',
 '20210810',
 '20210811',
 '20210812',
 '20210813',
 '20210814',
 '20210815',
 '20210816',
 '20210817',
 '20210818',
 '20210819',
 '20210820',
 '20210821',
 '20210822',
 '20210823',
 '20210824',
 '20210825',
 '20210826',
 '20210827',
 '20210828',
 '20210829',
 '20210830',
 '20210831',
 '20210901',
 '20210902',
 '20210903',
 '20210904',
 '20210905',
 '20210906',
 '20210907',
 '20210908',
 '20210909',
 '20210910',
 '20210911',
 '20210912',
 '20210913',
 '20210914',
 '20210915',
 '20210916',
 '20210917',
 '20210918',
 '20210919',
 '20210920',
 '20210921',
 '20210922',
 '20210923',
 '20210924',
 '20210925',
 '20210926',
 '20210927',
 '20210928',
 '20210929',

In [34]:
%%time 

# 1) Definición del query 
query_carga_real_general = f"""
select ZINVWMCAB.PURCH_NO_C, ZINVWMCAB.DOC_TYPE,  ZINVWMCAB.SALES_OFF as UNE, ZINVWMCAB.SALES_GRP as RUTA, ZINVWMCAB.PURCH_DATE, 
ZINVWMDET.MATERIAL, ZINVWMDET.TARGET_QTY from SAPABAP1.ZINVWMCAB
inner join SAPABAP1.ZINVWMDET on SAPABAP1.ZINVWMCAB.PURCH_NO_C = SAPABAP1.ZINVWMDET.PURCH_NO_C  
where ZINVWMCAB.PURCH_DATE IN {fechas_tuple}  
AND ZINVWMCAB.SALES_OFF IN {UNEs}
limit 1000
"""


# 2) Modificamos el formato 
query_carga_real_general = f"({query_carga_real_general}) as query_alias"

# 3) Lectura a un Spark DF 
carga_real_general = spark.read.jdbc(url=database_url_sap, table=query_carga_real_general, properties=database_properties_sap)

CPU times: total: 0 ns
Wall time: 735 ms


**GENERACIÓN DE UNA COLUMNA TIPO DATE**

In [36]:
carga_real_general = carga_real_general.withColumn("FECHA", to_date(col("PURCH_DATE"), "yyyyMMdd"))

**Agregamos una columna más legible para el tipo de carga** 

In [37]:
# 1) Lo convertimos a una Spark pandas DF 
carga_real_general = carga_real_general.to_pandas_on_spark() 

# 2) Creamos un diccionario 
tipo_carga  ={
    "CA": "Carga", 
    "PF": "Prefacturas", 
    "RE": "Recargas"
}

# 3) Hace el mapping al spark pandas DF  
carga_real_general["TIPO_OPERACION"] = carga_real_general["DOC_TYPE"].map(arg = tipo_carga)

**AGERGAMOS SEMANA, MES y AÑO**

In [38]:
# 3) Generamos el mes, año y semana apartir de la fecha con la funcion dt.. 
carga_real_general["Month"] = carga_real_general["FECHA"].dt.month
carga_real_general["Year"] = carga_real_general["FECHA"].dt.year
carga_real_general["Week"] = carga_real_general["FECHA"].dt.week
carga_real_general["Day"] = carga_real_general["FECHA"].dt.day

### 4.6) Volumen  y descripción  por productos 
En la tabla MARA viene cualquier tipo de prductos de la empresa, desde materiales para mantenmiento hasta los productos que se comercializan  

In [39]:
# 1) Definición del query 
query_productos = """
select MARA.MATNR as Material, MAKT.MAKTX as DESCRIPCION,  MARA.BRGEW as PesoTotal, MARA.NTGEW as PesoLiquido, MARA.GEWEI as Unidades 
from SAPABAP1.MARA  
inner join SAPABAP1.MAKT on SAPABAP1.MARA.MATNR = SAPABAP1.MAKT.MATNR   
where MTART = 'FERT'
"""


# La columna MTART, es el tipo de material, y los productos de venta son FERT de la columna MTAR

# NOTA: En la base de datos ST son piezas! 

# 2) Tranformación del formato query  
query_productos = f"({query_productos}) as query_alias"

# 3) Ejecución del query hacia un spark DF 
vol_productos = spark.read.jdbc(url=database_url_sap, table=query_productos, properties=database_properties_sap)

# 4) Tranformación hacia un spark pandas DF
vol_productos = vol_productos.to_pandas_on_spark()

,MATERIAL,DESCRIPCION,PESOTOTAL,PESOLIQUIDO,UNIDADES
0,000000000000000005,NECTAR DURAZNO UP 250ML,0.300,0.250,KG
1,000000000000000005,NECTAR DURAZNO UP 250ML,0.300,0.250,KG
2,000000000000000006,LECHE 3% GRASA PAST 1LT,1.160,1.000,KG
3,000000000000000006,3% GRASA 1LT PAST,1.160,1.000,KG
4,000000000000000007,NECTAR MANGO UP 250ML,0.300,0.250,KG
5,000000000000000007,NECTAR MANGO UP 250ML,0.300,0.250,KG
6,000000000000000009,NECTAR MANZANA UP 236ML,0.300,0.236,KG
7,000000000000000009,NECTAR MANZANA UP 236ML,0.300,0.236,KG
8,000000000000000012,NECTAR DURAZNO UP 236ML,0.300,0.236,KG
9,000000000000000012,NECTAR DURAZNO UP 236ML,0.300,0.236,KG


## 5) Tranformación de datos 

Migrar hacia esta sección cualquier manipulacion por más minima que sea , (los merge princiaplenten)

### 5.1) Agregar el volumen y descripción de prodcutos a las operaciones de venta y cambios físicos 


In [40]:
# Hacemos el merge de pedidos, ventas y cambios con su respectivo volumen por pieza 
ventas_pedidos_y_cambios = ventas_pedidos_y_cambios.merge(right=vol_productos, on=["MATERIAL"])

,PURCH_NO_C,DOC_TYPE,UNE,RUTA,PURCH_DATE,DIA_CLAVE,FORMA_PAGO,MATERIAL,TARGET_QTY,TOTAL_DETALLE,Month,Year,Week,Day,TIPO_OPERACION,METODO_PAGO,DESCRIPCION,PESOTOTAL,PESOLIQUIDO,UNIDADES
0,VEND00202000056854,ZPSR,D002,020,20240228,2024-02-28,1,000000000000000580,11.000,495.000000000,2,2024,9,28,Ventas,Efectivo,MEDIA CREMA UP 1LT,1.130,1.000,KG
1,VEND00202000056854,ZPSR,D002,020,20240228,2024-02-28,1,000000000000000580,11.000,495.000000000,2,2024,9,28,Ventas,Efectivo,MEDIA CREMA UP 1LT,1.130,1.000,KG
2,VEND00202000056859,ZPSR,D002,020,20240228,2024-02-28,1,000000000000000580,12.000,540.000000000,2,2024,9,28,Ventas,Efectivo,MEDIA CREMA UP 1LT,1.130,1.000,KG
3,VEND00202000056859,ZPSR,D002,020,20240228,2024-02-28,1,000000000000000580,12.000,540.000000000,2,2024,9,28,Ventas,Efectivo,MEDIA CREMA UP 1LT,1.130,1.000,KG
4,VEND00202000056876,ZPSR,D002,020,20240228,2024-02-28,1,000000000000000580,12.000,540.000000000,2,2024,9,28,Ventas,Efectivo,MEDIA CREMA UP 1LT,1.130,1.000,KG
5,VEND00202000056876,ZPSR,D002,020,20240228,2024-02-28,1,000000000000000580,12.000,540.000000000,2,2024,9,28,Ventas,Efectivo,MEDIA CREMA UP 1LT,1.130,1.000,KG
6,VEND00202000056879,ZPSR,D002,020,20240228,2024-02-28,1,000000000000000580,24.000,1080.000000000,2,2024,9,28,Ventas,Efectivo,MEDIA CREMA UP 1LT,1.130,1.000,KG
7,VEND00202000056879,ZPSR,D002,020,20240228,2024-02-28,1,000000000000000580,24.000,1080.000000000,2,2024,9,28,Ventas,Efectivo,MEDIA CREMA UP 1LT,1.130,1.000,KG
8,PAD00110120240228890051274,ZPAS,D001,101,20240228,2024-02-28,,000000000000000735,12.000,0E-9,2,2024,9,28,Pedidos_Planta,Sin información,SR DESLAC 1% UP BOLSA 900ML,0.900,0.900,KG
9,PAD00110120240228890051274,ZPAS,D001,101,20240228,2024-02-28,,000000000000000735,12.000,0E-9,2,2024,9,28,Pedidos_Planta,Sin información,SR DESLAC 1% UP BOLSA 900ML,0.900,0.900,KG


### 5.2) Agregamos una columna de meterial resumen 
Me refiero a solo un número sin los ceros a la izquierda 

In [41]:
# Tranformar la columna de material para que  quede  solo los ultimos numeros !
ventas_pedidos_y_cambios["METERIAL_RESUMEN"] = ventas_pedidos_y_cambios["MATERIAL"].astype(int)

,PURCH_NO_C,DOC_TYPE,UNE,RUTA,PURCH_DATE,DIA_CLAVE,FORMA_PAGO,MATERIAL,TARGET_QTY,TOTAL_DETALLE,Month,Year,Week,Day,TIPO_OPERACION,METODO_PAGO,DESCRIPCION,PESOTOTAL,PESOLIQUIDO,UNIDADES,METERIAL_RESUMEN
0,VEND00202000056854,ZPSR,D002,020,20240228,2024-02-28,1,000000000000000580,11.000,495.000000000,2,2024,9,28,Ventas,Efectivo,MEDIA CREMA UP 1LT,1.130,1.000,KG,580
1,VEND00202000056854,ZPSR,D002,020,20240228,2024-02-28,1,000000000000000580,11.000,495.000000000,2,2024,9,28,Ventas,Efectivo,MEDIA CREMA UP 1LT,1.130,1.000,KG,580
2,VEND00202000056859,ZPSR,D002,020,20240228,2024-02-28,1,000000000000000580,12.000,540.000000000,2,2024,9,28,Ventas,Efectivo,MEDIA CREMA UP 1LT,1.130,1.000,KG,580
3,VEND00202000056859,ZPSR,D002,020,20240228,2024-02-28,1,000000000000000580,12.000,540.000000000,2,2024,9,28,Ventas,Efectivo,MEDIA CREMA UP 1LT,1.130,1.000,KG,580
4,VEND00202000056876,ZPSR,D002,020,20240228,2024-02-28,1,000000000000000580,12.000,540.000000000,2,2024,9,28,Ventas,Efectivo,MEDIA CREMA UP 1LT,1.130,1.000,KG,580
5,VEND00202000056876,ZPSR,D002,020,20240228,2024-02-28,1,000000000000000580,12.000,540.000000000,2,2024,9,28,Ventas,Efectivo,MEDIA CREMA UP 1LT,1.130,1.000,KG,580
6,VEND00202000056879,ZPSR,D002,020,20240228,2024-02-28,1,000000000000000580,24.000,1080.000000000,2,2024,9,28,Ventas,Efectivo,MEDIA CREMA UP 1LT,1.130,1.000,KG,580
7,VEND00202000056879,ZPSR,D002,020,20240228,2024-02-28,1,000000000000000580,24.000,1080.000000000,2,2024,9,28,Ventas,Efectivo,MEDIA CREMA UP 1LT,1.130,1.000,KG,580
8,PAD00110120240228890051274,ZPAS,D001,101,20240228,2024-02-28,,000000000000000735,12.000,0E-9,2,2024,9,28,Pedidos_Planta,Sin información,SR DESLAC 1% UP BOLSA 900ML,0.900,0.900,KG,735
9,PAD00110120240228890051274,ZPAS,D001,101,20240228,2024-02-28,,000000000000000735,12.000,0E-9,2,2024,9,28,Pedidos_Planta,Sin información,SR DESLAC 1% UP BOLSA 900ML,0.900,0.900,KG,735


## 6) Carga de datos hacia GCP 

**Parámetros** 

In [27]:
print("Nombre del Bucket en Google Cloud Storage:", bucket)
print("Nombre del proyecto en GCP:", proyecto)
print("Conjunto de datos (base de datos) en Big Query:", conjunto_datos_BQ)
print("Método de inserción de datos en Big Query:", modo_escritura)
# WRITE_APPEND va a agregar datos a los ya existentes, WRITE_TRUNCATE reemplaza los datos existentes por los nuevos en BQ 

Nombre del Bucket en Google Cloud Storage: gs://sap_eroute
Nombre del proyecto en GCP: ciencia-de-datos-398421
Conjunto de datos (base de datos) en Big Query: SAP_Eroute
Método de inserción de datos en Big Query: WRITE_TRUNCATE


**DEFINICION del nombre de las tablas a exportar** 

In [28]:
tablas = {"agenda_actual_clientes_activos": agenda_actual_clientes_activos , 
          "agenda_historica": agenda_historica
          }

**EJECUCION DEL CICLO DE SUBIDA**

In [29]:
for i in tablas.keys():
    # 1) Exportación a Google Cloud Storage 
    gcs_path = f"{bucket}/{i}" # Direccion de exportacion 
    tablas[i].to_spark().write.parquet(path=gcs_path, mode="overwrite")  # Exportacion en modo overwrite para sobreescribir
    
    # 2) Identificamos la tabla dentro de Bigquery (sino existe la va crear) 
    table_id_BQ = f"{proyecto}.{conjunto_datos_BQ}.{i}"

    # Dirección de la fuente de datos (archivos parquet)
    uri = f"{gcs_path}/*" # con el asterisco indico que me traiga todos los parquetes de esa carepeta 
    
    # 3) Subida de datos a BQ 
    # Configurar el job_config, se asume que la estructura de los datos ya está definida en BigQuery
    job_config = bigquery.LoadJobConfig(source_format=bigquery.SourceFormat.PARQUET, 
                                    write_disposition = f"{modo_escritura}")  # Esto esta como write truncate por si existe lo borra 
    
    # Iniciar la  carga de datos a BigQuery
    load_job = client.load_table_from_uri(source_uris=uri, destination= table_id_BQ, job_config=job_config)  

    # Esperar a que el job se complete
    load_job.result()
    print(f"Se cargo la tabla {i}")

C:\Users\OMBARRAZA\miniconda3\envs\Spark\Lib\site-packages\pyspark\pandas\utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


Se cargo la tabla agenda_actual_clientes_activos
Se cargo la tabla agenda_historica
